In [1]:
import numpy as np
import netCDF4 as nc
import pandas as pd
import numpy.ma as ma

In [2]:
import matplotlib as mpl
import matplotlib.pyplot as plt

In [3]:
import xarray as xar

C:\ProgramData\Anaconda3\lib\site-packages\dask\config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
C:\ProgramData\Anaconda3\lib\site-packages\distributed\config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [4]:
file_path='./data/sanjioil.nc'

In [5]:
ds=nc.Dataset(file_path)

In [6]:
ds_xr=xar.open_dataset(file_path)

获取所有的维度  
* 注意获取维度时，不可以使用下标进行索引，会报错

## 获取`几个指定维度的`的`DataArray`  
如下可知：
    `mass_oil`有2958个值，该物理量有两个维度，分别为`trajectory`,`time`

#### 查看不同时刻的
并通过`where`剔除掉大于`max`以及`nan`的值  
使用xarray的dropna方法，注意与pandas中的方法有所区别

提示错误：
`ValueError: any must be a single dataset dimension`  
查看文档后发现`xarray.Dataset.dropna`尚不支持同时沿多个维度放置  

[参考](http://xarray.pydata.org/en/stable/generated/xarray.Dataset.dropna.html)  

考虑将xarray转换为dateframe，然后再清洗？  
[上网参考](https://stackoverflow.com/questions/52553925/python-xarray-remove-coordinates-with-all-missing-variables)
比较好的解决办法是：
1. 将`DataSet`->`DataFrame`
2. 在`DataFrame`中通过`pandas` `dropna` 即可

In [7]:
ds_xr.isel(time=30)['status']

<xarray.DataArray 'status' (trajectory: 2958)>
array([-2147483647,           0, -2147483647, ..., -2147483647, -2147483647,
       -2147483647])
Coordinates:
  * trajectory  (trajectory) int32 1 2 3 4 5 6 ... 2953 2954 2955 2956 2957 2958
    time        datetime64[ns] 2018-01-16T04:20:00
    lon         (trajectory) float32 ...
    lat         (trajectory) float32 ...
Attributes:
    valid_range:    [0 1]
    flag_values:    [0 1]
    flag_meanings:  active evaporated

In [8]:
ds_xr.isel(time=30)['mass_evaporated']

<xarray.DataArray 'mass_evaporated' (trajectory: 2958)>
array([9.969210e+36, 5.300872e+02, 9.969210e+36, ..., 9.969210e+36,
       9.969210e+36, 9.969210e+36], dtype=float32)
Coordinates:
  * trajectory  (trajectory) int32 1 2 3 4 5 6 ... 2953 2954 2955 2956 2957 2958
    time        datetime64[ns] 2018-01-16T04:20:00
    lon         (trajectory) float32 ...
    lat         (trajectory) float32 ...
Attributes:
    units:    kg

In [9]:
ds_xr.isel(time=30)['status']

<xarray.DataArray 'status' (trajectory: 2958)>
array([-2147483647,           0, -2147483647, ..., -2147483647, -2147483647,
       -2147483647])
Coordinates:
  * trajectory  (trajectory) int32 1 2 3 4 5 6 ... 2953 2954 2955 2956 2957 2958
    time        datetime64[ns] 2018-01-16T04:20:00
    lon         (trajectory) float32 ...
    lat         (trajectory) float32 ...
Attributes:
    valid_range:    [0 1]
    flag_values:    [0 1]
    flag_meanings:  active evaporated

对于未做任何处理的`dataarray`的len为`2958`  
使用max与min方法操作dataset  
`max`为1  
`min`为-2147xxxx

In [12]:
xr_temp_x=ds_xr.isel(time=30)['status']

In [13]:
xr_temp_x.max()

<xarray.DataArray 'status' ()>
array(1)
Coordinates:
    time     datetime64[ns] 2018-01-16T04:20:00

In [14]:
xr_temp_x.min()

<xarray.DataArray 'status' ()>
array(-2147483647)
Coordinates:
    time     datetime64[ns] 2018-01-16T04:20:00

In [28]:
ds_xr.isel(time=30)['lat'].mean().data

array(inf, dtype=float32)

## 1-先对dataarray进行过滤，先剔除<0的  
先剔除<0的部分后发现len并未减少？  
* 原因是看其中的array已经出现了nan的掩码，需要剔除

In [15]:
xr_temp_x.where(xr_temp_x>=0)

<xarray.DataArray 'status' (trajectory: 2958)>
array([nan,  0., nan, ..., nan, nan, nan])
Coordinates:
  * trajectory  (trajectory) int32 1 2 3 4 5 6 ... 2953 2954 2955 2956 2957 2958
    time        datetime64[ns] 2018-01-16T04:20:00
    lon         (trajectory) float32 9.96921e+36 125.85207 ... 9.96921e+36
    lat         (trajectory) float32 9.96921e+36 28.504345 ... 9.96921e+36
Attributes:
    valid_range:    [0 1]
    flag_values:    [0 1]
    flag_meanings:  active evaporated

In [16]:
# 再对status进行<1的筛选，当=1 时表示粒子死亡（到了落地——需要再看api）
xr_temp_x.where(xr_temp_x>=0).where(xr_temp_x<1)

<xarray.DataArray 'status' (trajectory: 2958)>
array([nan,  0., nan, ..., nan, nan, nan])
Coordinates:
  * trajectory  (trajectory) int32 1 2 3 4 5 6 ... 2953 2954 2955 2956 2957 2958
    time        datetime64[ns] 2018-01-16T04:20:00
    lon         (trajectory) float32 9.96921e+36 125.85207 ... 9.96921e+36
    lat         (trajectory) float32 9.96921e+36 28.504345 ... 9.96921e+36
Attributes:
    valid_range:    [0 1]
    flag_values:    [0 1]
    flag_meanings:  active evaporated

### dropna的方式1：直接对DataArray进行dropna  
* TypeError: dropna() missing 1 required positional argument: 'dim'

In [17]:
xr_filter=xr_temp_x.where(xr_temp_x>=0).dropna(dim='trajectory',how='any')
xr_filter.head()

<xarray.DataArray 'status' (trajectory: 5)>
array([0., 0., 0., 0., 0.])
Coordinates:
  * trajectory  (trajectory) int32 2 4 5 9 10
    time        datetime64[ns] 2018-01-16T04:20:00
    lon         (trajectory) float32 125.85207 125.83755 ... 125.94137
    lat         (trajectory) float32 28.504345 28.53567 ... 28.548067 28.543129
Attributes:
    valid_range:    [0 1]
    flag_values:    [0 1]
    flag_meanings:  active evaporated

In [18]:
# 查看一下长度
len(xr_filter)

901

### dropna的方式2：由于xarray中`DataArray`的dropna也是继承自`pandas`与`numpy`的  
* xarray objects borrow the isnull(), notnull(), count(), dropna(), fillna(), ffill(), and bfill() methods for working with missing data from pandas:   
 
[missing values](https://xarray-test.readthedocs.io/en/latest/computation.html)

In [19]:
len(xr_temp_x.where(xr_temp_x>=0).to_dataframe().dropna(how='any'))

901

## 2 求均值

In [20]:
xr_filter['lat'].mean()

<xarray.DataArray 'lat' ()>
array(28.47951, dtype=float32)
Coordinates:
    time     datetime64[ns] 2018-01-16T04:20:00

In [21]:
xr_filter['lon'].mean()

<xarray.DataArray 'lon' ()>
array(125.88219, dtype=float32)
Coordinates:
    time     datetime64[ns] 2018-01-16T04:20:00

In [22]:
xr_filter.mean()

<xarray.DataArray 'status' ()>
array(0.01664817)
Coordinates:
    time     datetime64[ns] 2018-01-16T04:20:00

获取均值结果  
对于`ndarray.data.tolist()`

In [23]:
# 获取DataArray的唯一值
xr_filter['lat'].mean().data.tolist()

28.479509353637695

In [24]:
xr_filter['lon'].mean().data.tolist()

125.88218688964844

In [39]:
xr_filter['lon'].mean().values

array(125.88219, dtype=float32)

In [69]:
xr_filter.mean().data.tolist()

0.016648168701442843